In [ ]:
#------ Prologue
import postgkyl as pg
import numpy as np
#import pylab as plt
#import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4)
import matplotlib.cm as cm
import os, fnmatch, re, sys, copy
#------ add personnal python utilities
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species
from utils import *
#------ Path towards the required folders
expdatadir ='/home/ah1032/amd/g0_sim/diagnostics/gk_tcv/tcv_exp_data/' # a possible directory for exp data
g0simdir   = home_dir+'/g0_sim/'        # This is the main directory of g0 simulations
simname    = '48x32x16x16x6_nunifvg'    # This is the name of the simulation
simdir     = 'gk_tcv_posD_iwl_3x2v_AH/' # This is the simulation directory relative to the g0sim
fileprefix = 'gk_tcv_posD_iwl_3x2v_D02' # This is the prefix before the '-*.gkyl'
wkdir      = 'wk/'                      # This is where the .gkyl files sit relatively to the sim dir
#----- some parameters of the simulation
#----- physical quantities
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.5,                 # Elongation factor
    delta       = 0.3,                 # Triangularity factor
    q0          = None,                 # Safety factor
    x_LCFS      = 0.04,                 # position of the LCFS (= core domain width)
    x_out       = 0.08                  # SOL domain width
)

# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
simulation.add_species(ion)
simulation.add_species(elc)

# Setup the data parameters, receipe and normalization of the fields
simulation.set_data_param(
    expdatadir = expdatadir, g0simdir = g0simdir, simname = simname, 
    simdir = simdir, fileprefix = fileprefix, wkdir = wkdir, 
    species = simulation.species)

# Load metric coefficients, magnetic field
simulation.geom_param.load_metric(simulation.data_param.fileprefix)

#----- set up the sources
simulation.set_OMPsources(
    n_srcOMP=2.4e23,
    x_srcOMP=0.0,
    Te_srcOMP=2 * simulation.species['elc'].T0,
    Ti_srcOMP=2 * simulation.species['ion'].T0,
    sigma_srcOMP=0.03 * simulation.geom_param.Lx,
    floor_src=1e-2,
)

# Check available frames of the ES potential
tfs = find_available_frames(simulation,'field')
print(tfs)

# ---- Set up normalization and units
simulation.normalize('t','mus')
simulation.normalize('x','minor radius')
simulation.normalize('y','Larmor radius')
simulation.normalize('z','pi')
simulation.normalize('fluid velocities','thermal velocity')
simulation.normalize('temperatures','eV')
simulation.normalize('pressures','beta')
simulation.normalize('energies','MJ')

In [ ]:
#---------------- Profile 
#-- Plane location
cdirection='x'; ccoords=['avg',0.0]
# cdirection='y'; ccoords=[0.01,0.0]
# cdirection='z'; ccoords=[0.01,'avg']
#----------------
plot_1D(simulation,cdirection,ccoords,
                 fieldnames=[
                     'phi',
                     ['ne','ni'],
                     ['Tpare','Tperpe'],['Te','Ti']
                     # ['upare','upari'],
                    #  'jpar',
                     ],
                 tfs=140,
                 )

In [ ]:
#---------------- Space-time diagrams
#-- Plane location
cdirection='x'; ccoords=[0.0,0.0]
# cdirection='y'; ccoords=[0.01,0.0]
# cdirection='ky'; ccoords=[0.01,0.0]
# cdirection='z'; ccoords=[0.01,'avg']
#----------------
plot_1D_time_evolution(simulation,cdirection,ccoords,
                       fieldnames = ['ne','upare','Ti','Te'], # field to plot (4 moments if nothing)
                       twindow    = tfs[::5], space_time = True
                       )

In [ ]:
#---------- 2D cut plot
#-- choose min and max time frames and cut location
cplane='xy'; ccoord = 0.0
plot_2D_cut(simulation, cplane, ccoord, 
            tf = tfs[-10], 
            # fieldnames = ['Wkine','Wflue','Wpote','Wtote'],
            fieldnames = ['ne','upare','Ti','phi'], # field to plot (4 moments if nothing)
            )

In [ ]:
#---- Comput and plot volume integral quantities (energies)
plot_volume_integral_vs_t(simulation,
                          # [['Wkin','Wflu','Wpot','Welf','Wtot'],
                          #  ['Wkine','Wkini'],
                          #  ['Wflue','Wflui'],
                          #  ['Wpote','Wpoti']],
                          fieldnames = [['Wtot']],
                          tfs=tfs[::10],
                          ddt = False, plot_src_input=True)

In [ ]:
#---- Comput and plot volume integral time-variation (energies)
plot_volume_integral_vs_t(simulation,
                        #   [['Wkin','Wflu','Wpot','Wtot'],
                        #    ['Wkine','Wkini'],
                        #    ['Wflue','Wflui'],
                        #    ['Wpote','Wpoti']],
                          fieldnames = 'Wtot',
                          tfs=tfs[::10],
                          ddt = True, jacob_squared=False, plot_src_input=True)

In [ ]:
#---- Compute particle loss due to fluctuations over time in the simulation
# (this does not depend on the srcGB parameters)
plot_GB_loss(simulation,tfs)

In [ ]:
#---------- 2D cut movie
#-- choose min and max time frames and cut location
cplane='xy'; ccoord=0.0
make_2D_movie(simulation, cplane, ccoord, 
            tfs = tfs[::10], 
            fieldname = 'Te',            
            # cmap = 'bwr'
            # clim=[0, 200],
            )

In [9]:
#---------- 2D cuts of a frame of the distribution function (long to load)
if False:
    # Simulation frame
    tf = tfs[-2]
    # Specie to plot (ion or elc)
    spec = elc
    # Point where each cuts intersect [in Gkeyll units or indices]
    xf =-0.03
    yf = 0.0
    zf = 0.0
    vf = 0.0
    mf = 0.0
    
    #---- Load the distf
    fname = simulation.get_filename('f'+spec.nshort,tf)
    data = pg.data.GData(fname)
    dg = pg.data.GInterpModal(data,1,'ms')
    dg.interpolate(0,overwrite=True)
    
    #---------------- 2D vpar x plots
    cut = copy.copy(data)
    pg.data.select(cut, z1=yf, z2=zf, z4=mf, overwrite=True)
    pg.output.plot(cut, yscale=1./spec.vt,
                title=r"y=%2.2f, z=%2.2f, $\mu$=%2.2f, t=%2.2e (s)"%(yf,zf,mf,data.ctx['time']),
                contour=False,diverging=False, xshift=-simulation.geom_param.x_LCFS,
                xlabel=r'$R-R_{lcfs}\,(m)$',ylabel=r'$v_\parallel/v_{th%s}$'%spec.nshort,
                clabel=r'$f_{%s}$'%spec.nshort);
    
    #---------------- 2D mu x plots
    cut = copy.copy(data)
    pg.data.select(cut, z1=yf, z2=zf, z3=vf, overwrite=True)
    pg.output.plot(cut, yscale=1./spec.mu0,
                title=r"y=%2.2f, z=%2.2f, $\mu$=%2.2f, t=%2.2e (s)"%(yf,zf,mf,data.ctx['time']),
                contour=False,diverging=False, xshift=-simulation.geom_param.x_LCFS,
                xlabel=r'$R-R_{lcfs}\,(m)$',ylabel=r'$\mu B/T_{%s}$'%spec.nshort,
                clabel=r'$f_{%s}$'%spec.nshort);
    
    #---------------- 2D vpar mu plots
    cut = copy.copy(data)
    pg.data.select(cut, z0=xf, z1=yf, z2=zf, overwrite=True)
    pg.output.plot(cut, yscale=1./spec.mu0,
                title=r"x=%2.2f, y=%2.2f, z=%2.2f, t=%2.2e (s)"%(xf,yf,zf,data.ctx['time']),
                contour=False,diverging=False, xshift=-simulation.geom_param.x_LCFS,
                xlabel=r'$v_\parallel/v_{th%s}$'%spec.nshort,ylabel=r'$\mu B/T_{%s}$'%spec.nshort,
                clabel=r'$f_{%s}$'%spec.nshort);